# Program to fit a straight line to the set of arbitrary strips

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
class RANSAC:
    def __init__(self, n=10, k=100, t=0.05, d=10, model=None, loss=None, metric=None):
        self.n = n              # `n`: Minimum number of data points to estimate parameters
        self.k = k              # `k`: Maximum iterations allowed
        self.t = t              # `t`: Threshold value to determine if points are fit well
        self.d = d              # `d`: Number of close data points required to assert model fits well
        self.loss = loss        # `loss`: function of `y_true` and `y_pred` that returns a vector
        self.metric = metric    # `metric`: function of `y_true` and `y_pred` and returns a float
        self.fun = 0
        self.x = np.zeros(4)
        self.jac = np.zeros(4)
        
    # calculate Chi2
    def chiSquare(self, par):
        # par = [Ax, Ay, Bx, By]
        # track:  X = Ax*z +Bx, Y = Ay*z +By
        #  
        
        Z0 = 0
        Z1 = 100
        tr0 = (par[0]*Z0+par[2], par[1]*Z0+par[3], Z0)
        tr1 = (par[0]*Z1+par[2], par[1]*Z1+par[3], Z1)
           
        chi2 = 0
        for iStrip in strips:
            chi2 = chi2 + self.distance_between_lines(tr0, tr1, iStrip[0], iStrip[1])**2
            #print("chi2 ",tr0, tr1, iStrip[0], iStrip[1], distance_between_lines(tr0, tr1, iStrip[0], iStrip[1]))
        return chi2           

    # calculate distance between two lines in 3D
    def distance_between_lines(self, a, b, c, d):
        # a - first point of the first line in 3D (x,y,z)
        # b - second point of the first line
        # c - first point of the second line
        # d - second point of the second line
        # https://math.stackexchange.com/questions/210848/finding-the-shortest-distance-between-two-lines
        v1 = np.array(b) - np.array(a)
        v2 = np.array(d) - np.array(c)
        cross = np.cross(v1, v2)
        if cross.any() != 0:
            # lines not parallel
            cross = cross/np.linalg.norm(cross)
            return np.dot(cross,np.subtract(c,a))
        else:
            # lines parallel - calculate a distance from point c to line (a,b)
            # https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points
            return np.linalg.norm(np.cross(np.subtract(b,a),np.subtract(c,a)))/np.linalg.norm(np.subtract(b,a))
        
    #actual fit    
    def fit(self, iStrips):
        
        par = np.zeros(4)
        
        #xx = self.chiSquare(self)
       
        res = minimize(self.chiSquare, par, method='SLSQP')
        self.x = res.x
        self.jac = res.jac
        self.fun = res.fun
        print(res)
              
        return self

# Example of  usage

In [3]:
# Fit a straight track (described by pars) to the set of arbitrary strips (described by strips)

strips = ( ((0, 0, 0), (1, 1, 0)), ((0, 1, 1), (1, 0, 1)),
           ((0, 0, 5), (1, 1, 5)), ((0, 1.2, 6), (1.2, 0, 6)),
           ((0, 0, 10), (1, 1, 10)), ((0, 1, 11), (1, 0, 11)))
#strips = (((0, 0, 0), (0, 0, 3)), ((1, 1, 0), (1, 1, 30)))

regressor = RANSAC()
regressor.fit(strips)


print("Fitted chi_2: ",regressor.fun, "Fit results: ",regressor.x," +- ", regressor.jac)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.013333334520843709
       x: [-3.365e-06 -2.958e-06  5.333e-01  5.333e-01]
     nit: 6
     jac: [-2.238e-03  1.726e-03 -2.429e-04  2.373e-04]
    nfev: 38
    njev: 6
Fitted chi_2:  0.013333334520843709 Fit results:  [-3.36494863e-06 -2.95841841e-06  5.33346838e-01  5.33346891e-01]  +-  [-0.00223782  0.00172553 -0.00024288  0.00023727]
